# Actualizacion de informacion de MyHandling

### 1ra etapa:
##### El bot elimina todos los archivos de la carpeta de descargas

In [44]:
import os
 
dir = 'C:/Users/Pedro Vallarino/Downloads'
for f in os.listdir(dir):
    os.remove(os.path.join(dir, f))

### 2da etapa:
##### El bot descarga el excel de flight paths automaticamente

In [45]:
#%pip install selenium
#%pip install webdriver_manager

In [46]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# Initiate the browser
browser  = webdriver.Chrome(ChromeDriverManager().install())

# Open the Website
browser.get('https://ap.myhandlingsoftware.com/Account/Login?ReturnUrl=%2f')

#Login info
myHandling_email = 'David.vallve@skyvalet.com'
myHandling_password = '88888888'

#Input password
browser.find_element(By.XPATH,'//*[@id="UserName"]').send_keys(myHandling_email)
browser.find_element(By.XPATH,'//*[@id="Password"]').send_keys(myHandling_password)
browser.find_element(By.XPATH,'/html/body/div[2]/div/div[5]/div/div[1]/div/div[1]/form/div[3]/div/input').click()

#Redirigo a Flight Paths y descargo el excel
browser.get('https://ap.myhandlingsoftware.com/Home/BoardList')
browser.find_element(By.XPATH,'//*[@id="ExportLinkExcel"]').click()

import os.path
import time

while not os.path.exists("C:/Users/Pedro Vallarino/Downloads/Flights.xlsx"):
    time.sleep(1)

browser.close()

C:\Users\Pedro Vallarino\AppData\Local\Temp\ipykernel_23724\3529882507.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser  = webdriver.Chrome(ChromeDriverManager().install())


### 3ra etapa:
##### El bot realiza los siguientes pasos:
1) Elimina el archivo Flights.xslx del directorio
2) Corta y pega el archivo Flights.xslx recientemente descargado desde Descargas hacia el directorio 

In [47]:
try: 
    os.remove("C:/Users/Pedro Vallarino/Desktop/Repos/Python/MyHandling Bot/Flights.xlsx")
except:
    os.path.exists("C:/Users/Pedro Vallarino/Desktop/Repos/Python/MyHandling Bot/Flights.xlsx") is False    

import shutil

shutil.move("C:/Users/Pedro Vallarino/Downloads/Flights.xlsx", "C:/Users/Pedro Vallarino/Desktop/Repos/Python/MyHandling Bot/Flights.xlsx")

'C:/Users/Pedro Vallarino/Desktop/Repos/Python/MyHandling Bot/Flights.xlsx'

### 4ta etapa:
##### El bot sube esta informacion de la carpeta de descargas a el drive

In [48]:
#%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
#%pip install Google

In [49]:
import pickle
from google_auth_oauthlib.flow import Flow, InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request

def Create_Service(client_secret_file, api_name, api_version, *scopes):
    print(client_secret_file, api_name, api_version, scopes, sep='-')
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    print(SCOPES)

    cred = None

    pickle_file = f'token_{API_SERVICE_NAME}_{API_VERSION}.pickle'
    # print(pickle_file)

    if os.path.exists(pickle_file):
        with open(pickle_file, 'rb') as token:
            cred = pickle.load(token)

    if not cred or not cred.valid:
        if cred and cred.expired and cred.refresh_token:
            cred.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            cred = flow.run_local_server()

        with open(pickle_file, 'wb') as token:
            pickle.dump(cred, token)

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=cred)
        print(API_SERVICE_NAME, 'service created successfully')
        return service
    except Exception as e:
        print('Unable to connect.')
        print(e)
        return None

La primera vez se crea el archivo, a partir de ahi vamos a correr el proximo

In [51]:
CLIENT_SECRET_FILE = 'credentials.json'
API_NAME = 'drive'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/drive']

Service = Create_Service(CLIENT_SECRET_FILE, API_NAME, API_VERSION, SCOPES)

folder_id = '1BxVBV0UqNr53kCcGUWqAPiXk3bQYrQR7'
file_names = ['Flights.xlsx']
mime_types = ['application/vnd.openxmlformats-officedocument.spreadsheetml.sheet']

for file_name, mime_type in zip(file_names, mime_types):
    file_metadata = {
        'name': file_name,
        'parents': [folder_id]
    }

media = MediaFileUpload('./{0}'.format(file_name), mimetype=mime_type )

Service.files().create(
    body = file_metadata,
    media_body = media,
    fields = 'id'
).execute()

credentials.json-drive-v3-(['https://www.googleapis.com/auth/drive'],)
['https://www.googleapis.com/auth/drive']
drive service created successfully


{'id': '1Jhb7vzTmzWz03_M6Psed_O2NKM6qnGEp'}

De la segunda en adelante se corre esta

In [52]:
mime_types = ['application/vnd.openxmlformats-officedocument.spreadsheetml.sheet']
file_id = '1c3cK2BK0paAe-cAAjj1pJh2WYUdwd0Af'
media_content = MediaFileUpload('Flights.xlsx', mimetype=mime_type )

Service.files().update(
    fileId = file_id,
    media_body = media_content,
).execute()

{'kind': 'drive#file',
 'id': '1c3cK2BK0paAe-cAAjj1pJh2WYUdwd0Af',
 'name': 'Flights.xlsx',
 'mimeType': 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'}